# Fixed Effects: Indicator Variables for Groups

One common use of indicator variables are as *fixed effects*. Fixed effects are used when our data as a "nested" structure (we think individual observations belong to groups), and we suspect different things may be happening in each group. 

For example, we might have a dataset of student test scores, and we may suspect that all students at a given school will tend to have higher or lower scores by virtue of being at that school. Or we might have data on earnings and gender, and we may think that incomes are likely to vary across industries or different cities. 

In this regard, fixed effects are analogous in purpose to hierarchical models, though they are slightly different in implementation (differences between fixed effects and hierarchical models are [discussed here](fixed_effects_v_hierarchical.ipynb)). 

In their simplest form, fixed effects can be included in a model just by adding indicator variables for each group. To illustrate, we'll use data from the US Current Population Survey (CPS) on US wages in 2019:

In [1]:
import pandas as pd

# Load survey
cps = pd.read_stata('https://github.com/nickeubank/MIDS_Data/blob/master/Current_Population_Survey/morg18.dta?raw=true')

# Limit to people currently employed and working full time. 
cps = cps[cps.lfsr94 == 'Employed-At Work']
cps = cps[cps.uhourse >= 35]

# And we can adjust earnings per hour (in cents) into dollars, 
cps['earnhre_dollars'] = cps['earnhre'] / 100
cps['annual_earnings'] = cps['earnhre_dollars'] * cps['uhourse'] * 52

# And create gender and college educ variable
cps['female'] = (cps.sex == 2).astype('int')
cps['has_college_educ'] = (cps.grade92 > 43).astype('int')

To illustrate the use of fixed effects, we begin with a simple model of earnings:

In [2]:
import statsmodels.formula.api as smf
smf.ols('annual_earnings ~ female + age + has_college_educ', cps).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        annual_earnings   R-squared:                       0.102
Model:                            OLS   Adj. R-squared:                  0.102
Method:                 Least Squares   F-statistic:                     2490.
Date:                Sun, 16 Feb 2020   Prob (F-statistic):               0.00
Time:                        10:03:57   Log-Likelihood:            -7.5063e+05
No. Observations:               65755   AIC:                         1.501e+06
Df Residuals:                   65751   BIC:                         1.501e+06
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept         3.239e+04    281.264    115.142      0.000    3.18e+04    3.29e+04
female           -8052.8166    172.035    -46.809      0.000   -8390.004   -7715.629
age                290.0363      6.255     46.370      0.000     277.777     302.296
has_college_educ  2.367e+04    410.579     57.646      0.000    2.29e+04    2.45e+04
==============================================================================
Omnibus:                    31647.447   Durbin-Watson:                   1.882
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           264260.027
Skew:                           2.152   Prob(JB):                         0.00
Kurtosis:                      11.828   Cond. No.                         209.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In this model, we're getting estimates of how education and gender explain variation across all Americans. 

But in this dataset, we also have a variable that tells us the state and county in which each respondent resides. Suppose we think that compensation may vary across regions of the country, and may also be correlated with things like whether a person has a college education. New York, for example, has a strong economy, and more people in NY have college degrees than in the average US city. Grand Forks, North Dakota, by contrast, probably has lower pay and lower education levels. Thus if we don't take into account geography, our estimate of the value of education will likely be overstated since we're capturing not only how education correlates with income, but also the fact that people with college degrees tend to live in towns that pay better. 

To correct for this, we can add an indicator for every county in the US to our model:

In [3]:
# The county variable is an id for counties *within*
# each state, so we have to concatenate county ID numbers
# with state names to get unique values for every actual 
# county in the US. 

cps['county_code'] = cps.stfips + '_county_' + cps.county.astype('str')

Then we can run the following regression:

In [4]:
# Adding semi-colon to suppress output because too long...
smf.ols('annual_earnings ~ female + age + has_college_educ + C(county_code)', cps).fit().summary();

which will generate output that will look approximately like this (note your output will be VERY long -- I'm omitting all the county coeffients for space. We'll talk later about how to suppress those in your output):

![cps_fe_unclustered_p1](images/cps_fe_unclustered_p1.png)
```
               .
               .
               .
```
![cps_fe_unclustered_p2](images/cps_fe_unclustered_p2.png)

Voila! What you've just estimated is no longer the relationship between education and income across all Americans, but rather the relationship between education and income *within in each US county*. 

To be clear, fixed effects aren't *mathematically* different from adding a normal control variable. One could say that adding `female` means that we're now estimating the relationship between education and income among men and among women. *Mechnically*, fixed effects are just additional indicator variables. But because we often use them for groups, thinking about the fact that, when added, one is effectively estimating variation *within* the groups specified by the fixed effects is a powerful idea. 

Perhaps no place is this more clear than in full panel data, where you have data on the same entities over time. In a panel regression, the addition of entity fixed effects allow you to difference out any *constant* differences between entities, and focus only on changes within each entity over time. This even works for people! In a panel with individuals observed over time, adding individual fixed effects means you're effectively controlling for anything constant about each individual (things that don't change over time), and now you're just studying *changes over time* for each individual. 

## Clustering

When working with fixed effects, however, it's also often a good idea to cluster your standard errors by your fixed effect variable. Clustering is a method for taking into account some of the variation in your data isn't coming from the individual level (where you have lots of observations), but rather from the group level. Since you have fewer groups than observations, clustering corrects your standard errors to reflect the smaller effective sample size being used to estimate those fixed effects (clustering *only* affects standard errors -- it has no impact on coefficients themselves. This is just about adjustments to our confidence in our inferences). 

Clustering is thankfully easy to do -- just use the `get_robustcov_results` method from `statsmodels`, and use the `groups` keyword to pass the group assignments for each observation. 

**Note that if you're using formulas in statsmodels, the regression is automatically dropping observations that can't be estimated because of missing data, so you have to do the same before passing your group assignments to** `get_robustcov_results` **-- otherwise you'll get the error:**

```
ValueError: The weights and list don't have the same length.
```

**because the number of observations in the model doesn't match the number of observations in the group assignment vector you pass!**

In [5]:
model = smf.ols('annual_earnings ~ female + age + has_college_educ + C(county_code)', cps).fit()

# Drop any entries with missing data from the model
fe_groups = cps.copy()
for i in ['annual_earnings', 'female', 'age', 'county_code', 'has_college_educ']:
    fe_groups = fe_groups[pd.notnull(fe_groups[i])]

In [6]:
# Adjust SEs
# Again, suppressing actual output for space with semicolon
model.get_robustcov_results(cov_type='cluster', 
                            groups=fe_groups.county_code).summary();

/Users/Nick/miniconda3/lib/python3.7/site-packages/statsmodels/base/model.py:1752: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 331, but rank is 3
  'rank is %d' % (J, J_), ValueWarning)


![cps_fe_clustered_p1](images/cps_fe_clustered_p1.png)
```
               .
               .
               .
```
![cps_fe_clustered_p2](images/cps_fe_clustered_p2.png)

As you can see, while our point estimates haven't changed at all (the coefficient on `female`, for example, is still -7,923), we have increased the size of our standard errors. The SE on `female`, for example, has gone from 171 without clustering to 273 with clustering.

## PanelOLS

OK, so everything we've describe up till here is a reasonable approach to fixed effects, but it has two limitations: our regression output looks *terrible*, and computing all those intercepts was slow.

This brings us to some of the specialized methods for calculating fixed effects. It turns out that if you aren't interested in the coefficient on each fixed effect, there are much more computationally efficient methods of calculating fixed effects. But to use them, we'll have to use a different library: [linearmodels](https://bashtage.github.io/linearmodels/doc/index.html) (installable using `pip install linearmodels`).

(R users: 

In particular, we'll be using the `PanelOLS` function from `linearmodels`. As the name implies, `PanelOLS` is designed for linear regression (social scientists call linear regression Ordinary Least Squares, or OLS) with panel data, which is really any form of data organized along two dimensions. Normally a panel has data on many entities observed several times, so the first dimension is the `entity` dimension, and the second is the `time` dimension. 

In this case, we don't really have a panel -- just nested data -- but because fixed effects are commonly used in panels, we'll use this tool. 

The only catch is: you have to use `multiindexes` in `pandas`. I *know*, I hate them too. But the multi-index is required by the library for it to understand what variable constitutes the "group" for which you want to add fixed effects. Basically `PanelOLS` calls the first level of the multi-index the `entity` and the second level `time`. In this case, though, we'll just make the first level our counties, and the second level individual identifiers, then use `entity` fixed effects (and clustering). 

In [7]:
cps.head()

county  smsastat  age  sex  grade92  race  ethnic  marital  uhourse  \
2        0       1.0   52    2       39     2     NaN        5     40.0   
3        0       1.0   19    2       39     2     NaN        7     40.0   
4        0       1.0   56    2       43     2     NaN        5     40.0   
6       97       1.0   48    1       39     1     NaN        7     40.0   
17      97       1.0   59    1       39     2     NaN        7     40.0   

    earnhre  ...            lfsr94               class94  unioncov  \
2    2084.0  ...  Employed-At Work    Government - State             
3    1000.0  ...  Employed-At Work   Private, For Profit        No   
4    2500.0  ...  Employed-At Work  Government - Federal             
6    1700.0  ...  Employed-At Work   Private, For Profit        No   
17   2000.0  ...  Employed-At Work   Private, For Profit        No   

                                                ind02  stfips  \
2   Residential care facilities, without nursing (...      AL   
3                    Business support services (5614)      AL   
4                                     Hospitals (622)      AL   
6                          Truck transportation (484)      AL   
17  ****Department stores and discount stores (s45...      AL   

    earnhre_dollars  annual_earnings  female  has_college_educ   county_code  
2             20.84          43347.2       1                 0   AL_county_0  
3             10.00          20800.0       1                 0   AL_county_0  
4             25.00          52000.0       1                 0   AL_county_0  
6             17.00          35360.0       0                 0  AL_county_97  
17            20.00          41600.0       0                 0  AL_county_97  

[5 rows x 30 columns]

In [8]:
# Move county groups into highest level of multi-index, 
# with old index in second level. 
# PanelOLS will then see the first level as the `entity` 
# identifier. 
cps_w_multiindex = cps.set_index(['county_code', cps.index])
cps_w_multiindex.head()

county  smsastat  age  sex  grade92  race  ethnic  marital  \
county_code                                                                   
AL_county_0  2        0       1.0   52    2       39     2     NaN        5   
             3        0       1.0   19    2       39     2     NaN        7   
             4        0       1.0   56    2       43     2     NaN        5   
AL_county_97 6       97       1.0   48    1       39     1     NaN        7   
             17      97       1.0   59    1       39     2     NaN        7   

                 uhourse  earnhre  ...  occ2012            lfsr94  \
county_code                        ...                              
AL_county_0  2      40.0   2084.0  ...   5700.0  Employed-At Work   
             3      40.0   1000.0  ...   5240.0  Employed-At Work   
             4      40.0   2500.0  ...   3255.0  Employed-At Work   
AL_county_97 6      40.0   1700.0  ...   9130.0  Employed-At Work   
             17     40.0   2000.0  ...   9620.0  Employed-At Work   

                              class94  unioncov  \
county_code                                       
AL_county_0  2     Government - State             
             3    Private, For Profit        No   
             4   Government - Federal             
AL_county_97 6    Private, For Profit        No   
             17   Private, For Profit        No   

                                                             ind02  stfips  \
county_code                                                                  
AL_county_0  2   Residential care facilities, without nursing (...      AL   
             3                    Business support services (5614)      AL   
             4                                     Hospitals (622)      AL   
AL_county_97 6                          Truck transportation (484)      AL   
             17  ****Department stores and discount stores (s45...      AL   

                 earnhre_dollars  annual_earnings  female  has_college_educ  
county_code                                                                  
AL_county_0  2             20.84          43347.2       1                 0  
             3             10.00          20800.0       1                 0  
             4             25.00          52000.0       1                 0  
AL_county_97 6             17.00          35360.0       0                 0  
             17            20.00          41600.0       0                 0  

[5 rows x 29 columns]

In [9]:
from linearmodels import PanelOLS
mod = PanelOLS.from_formula('annual_earnings ~ female + age + has_college_educ + EntityEffects', 
                            data=cps_w_multiindex)
mod.fit(cov_type='clustered', cluster_entity=True)

/Users/Nick/miniconda3/lib/python3.7/site-packages/linearmodels/utility.py:478: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)


PanelOLS Estimation Summary                           
================================================================================
Dep. Variable:        annual_earnings   R-squared:                        0.1000
Estimator:                   PanelOLS   R-squared (Between):              0.3898
No. Observations:               65755   R-squared (Within):               0.1000
Date:                Sun, Feb 16 2020   R-squared (Overall):              0.3265
Time:                        10:04:03   Log-likelihood                -7.499e+05
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      2422.5
Entities:                         329   P-value                           0.0000
Avg Obs:                       199.86   Distribution:                 F(3,65423)
Min Obs:                       2.0000                                           
Max Obs:                       3219.0   F-statistic (robust):             937.96
                                        P-value                           0.0000
Time periods:                   65755   Distribution:                 F(3,65423)
Avg Obs:                       1.0000                                           
Min Obs:                       1.0000                                           
Max Obs:                       1.0000                                           
                                                                                
                                Parameter Estimates                                 
====================================================================================
                  Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------------
female              -7923.2     272.07    -29.122     0.0000     -8456.4     -7389.9
age                  287.28     7.6942     37.336     0.0000      272.20      302.36
has_college_educ  2.313e+04     768.88     30.086     0.0000   2.163e+04   2.464e+04
====================================================================================

F-test for Poolability: 4.4314
P-value: 0.0000
Distribution: F(328,65423)

Included effects: Entity
PanelEffectsResults, id: 0x7facca3dad68

As you can see, the coefficients of the `PanelOLS` model are exactly the same as those we calculated above, and the standard errors are nearly identical (there are a few ways to calculate clustered standard errors, so they aren't numerically equivalent). But the way `PanelOLS` added fixed effects was much more computationally efficient efficient, and in these situations, we don't usually want to see the coefficients, so they're suppressed. 